In [3]:
from __future__ import print_function


class Event(object):
    pass

In [4]:
class MarketEvent(Event):
    def __init__(self):
        self.type = "MARKET"
    
    
class SignalEvent(Event):
    def __init__(self, strategy_id, symbol, datetime, singal_type, strength):
        self.type = "SIGNAL"
        self.strategy_id = strategy_id
        self.datetime = datetime
        slef.signal_type = signal_type
        self.strength = strength
        
class OrderEvent(Event):
    def __init__(self, symbol, order_type, quantity, direction):
        self.type = "ORDER"
        self.symbol = symbol
        self.order_type = order_type
        self.quantity = quantity
        self.direction = direction
        
    def print_order(self):
        print(
            "Order: Symbol=%s, Type=%s,  Quantity=%s, Direction=%s" % 
            (self.symbol, self.order_type, self.quantity, self.direction)
        )
        
class FillEvent(Event):
    def __init__(self,  timeindex, symbol, exchange, quantity,
                 direction, fill_cost, commission=None):
        self.type = "FILL"
        self.timeindex = timeindex
        self.symbol = symbol
        self.exchange = exchange
        self.quantity = quantity
        self.direction = direction
        self.fill_cost = fill_cost
        
        if commission is None:
            self.commission = self.caculate_ib_commission()
        else:
            self.commission = commission
            
    def calculate_ib_commission(self):
        full_cost = 1.3
        if self.quantity <= 500:
            full_cost = max(1.3, 0.013 * self.quantity)
        else:
            full_cost = max(1.3, 0.008 * self.quantity)
        return full_cost

In [5]:
from __future__ import print_function

from abc import ABCMeta, abstractmethod
import datetime
import os, os.path
import numpy as np
import pandas as pd

In [7]:
class DataHandler(metaclass=ABCMeta):
    @abstractmethod
    def  get_latest_bar(self, symbol):
        raise NotImplementedError()
        
    @abstractmethod
    def get_latest_bars(self, symbol, N=1):
        raise NotImplementedError()
    
    @abstractmethod
    def get_latest_bar_datetime(self, symbol):
        raise NotImplementedError()
        
    @abstractmethod
    def get_latest_bar_value(self, symbol, val_type):
        raise NotImplementedError()
        
    @abstractmethod
    def get_latest_bars_value(self, symbol, val_type, N=1):
        raise NotImplementedError()
        
    @abstractmethod
    def update_vars(self):
        raise NotImplementedError()

In [16]:
import pandas as pd

class HistoricCSVDataHandler(DataHandler):
    def __init__(self,  events, csv_dir, symbol_list):
        self.events = events
        self.csv_dir = csv_dir
        self.symbol_list = symbol_list
        
        self.symbol_data = {}
        self.latest_symbol_data = {}
        self.continue_backtest = True
        
        self._open_convert_csv_files()
        
    def _open_convert_csv_files(self):
        comb_index = None
        for s in self.symbol_list:
            self.symbol_data[s] = pd.io.parsers.read_csv(
                os.path.join(self.csv_dir, "%s.csv" % s),
                header=0, index_col=0, parse_dates=True,
                names=[
                    "datetime", "open", "high",
                    "low", "close", "volume", "adj_close",
                ]
            ).sort()
            
            if comb_index is None:
                comb_index = self.symbol_data[s].index
            else:
                comb_index.union(self.symbol_data[s].index)
            
            # Set the latest symbol_data to None
            self.latest_symbol_data[s] = []
        
        # Reindex the dataframes
        for s in self.symbol_list:
            self.symbol_data[s] = slef.symbol_data[s].\
                reindex(index=comb_index, method='pad').iterrows()
                
    def _get_new_bar(self, symbol):
        for b in self.symbol_data[symbol]:
            yield b
            
    def get_latest_bar(self, symbol):
        try:
            bars_list = self.latest_symbol_data[symbol]
        except KeyError:
            print("That symbol is not available in the historical data")
            raise
        else:
            return bars_list[-1]
        
    def get_latest_bars(self, symbol, N=1):
        try:
            bars_list = self.latest_symbol_data[symbol]
        except KeyError:
            print("That symbol is not available in the historical data")
            raise
        else:
            return bars_list[-N:]
        
    def get_latest_bar_datetime(self,  symbol):
        try:
            bars_list = self.latest_symbol_data[symbol]
        except KeyError:
            print("That symbol is not avaialable in the historical data")
            raise
        else:
            return bars_list[-1][0]
        
    def get_latest_bar_value(self, symbol, val_type):
        try:
            bars_list = self.latest_symbol_data[symbol]
        except KeyError:
            print("That symbol is not avaialble in the historical data")
            raise
        else:
            return getattr(bars_list[-1][1], val_type)
        
    def get_latest_bar_values(self, symbol, val_type, N=1):
        try:
            bars_list = self.latest_symbol_data[symbol]
        except KeyError:
            print("That symbol is not available in the historical data")
            raise
        else:
            return np.array([getattr(b[1], val_type) for b in bars_list])
        
    def update_bars(self):
        for s in self.symbol_list:
            try:
                bar = next(self._get_new_bar(s))
            except StopIteration:
                self.continue_backtest = False
            else:
                if bar is not None:
                    self.latest_symbol_data[s].append(bar)
        self.events.put(MarketEvent())

In [18]:
from __future__ import print_function

from abc import ABCMeta, abstractmethod
import datetime
import queue

import numpy as np
import pandas as pd

In [20]:
class Strategy(metaclass=ABCMeta):
    @abstractmethod
    def calculate_signals(self):
        raise NotImplementedError()

In [22]:
def create_sharpe_ratio(returns, periods=252):
    return np.sqrt(periods) * np.mean(returns) / np.std(returns)

def create_drawdowns(pnl):
    hwm = [0]
    idx = pnl.index
    drawdown = pd.Series(index=idx)
    duration = pd.Series(index=idx)
    
    for t in range(1, leen(idx)):
        hwm.append(hwm[t-1], pnl[t])
        drawdown[t] = hwm[t] - pnl[t]
        duration[t] = 0 if drawdown[t] == 0 else duration[t-1]+1
    return drawdown, drawdown.max, duration.max()

In [24]:
class Portfolio(object):
    def __init__(self,  bars, events, start_date, initial_capital=10000.0):
        self.bars = bars
        self.events = events
        self.symbol_list = symbol_list
        self.start_date = start_date
        self.initial_capital = initial_capital
        
        self.all_positions = self.construct_all_positions()
        self.current_positions = dict((k, v) for k,  v  in [(s, 0) for s in self.symbol_list])
        
        
        self.all_holdings = self.construct_all_holdings()
        self.current_holdings = self.construct_current_holdings()
        
    def construct_all_positions(self):
        d = dict((k, v) for k, v  in [(s,  0) for s in self.symbol_list])
        d["datetime"] = self.start_date
        return [d]
    
    def construct_all_holdings(self):
        d = dict((k, v) for k, v in [(s, 0) for s in self.symbol_list])
        d["datetime"] = self.start_date
        d["cash"] = self.initial_capital
        d["commission"] = 0.0
        d["total"] = self.initial_capital
        return [d]
    
    def construct_current_holdings(self):
        d = dict((k, v) for k, v in [(s, 0.0) for s in self.symbol_list])
        d["cash"] = self.initial_capital
        d["commission"] = 0.0
        d["total"] = self.initial_capital
        return d
    
    def update_timeindex(self,  event):
        dp  = dict((k, v) for k, v in [(s, 0) for s in self.symbol_list])
        dp["datetime"] = latest_datetime
        
        for s in self.symbol_list:
            dp[s] = self.current_positions[s]
            
        self.all_positions.append(dp)
        
        dh = dict((k, v) for k, v in [(s, 0) for s in self.symbol_list])
        dh["datetime"] = latest_datetime
        dh["cash"] = self.current_hodlings["cash"]
        dh["commission"] = self.current_holdings["commission"]
        dh["total"] = self.current_hodlings["cash"]
        
        for s in  self.symbol_list:
            market_value = self.current_positions[s] *\
                self.bars.get+latest_bar_values(s, "adj_close")
            dh[s] = market_value
            dh["total"] += market_value
        
        self.all_holdings.append(dh)

In [15]:
for i in df.iterrows():
    print(i)
    print(type(i))

(0, c1     10
c2    100
Name: 0, dtype: int64)
<class 'tuple'>
(1, c1     11
c2    110
Name: 1, dtype: int64)
<class 'tuple'>
(2, c1     12
c2    120
Name: 2, dtype: int64)
<class 'tuple'>
